In [1]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" spacy==3.1.1

Looking in links: ../input/commonlit-pkgs-offline-download
Processing /kaggle/input/commonlit-pkgs-offline-download/spacy-3.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/pathy-0.6.0-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/catalogue-2.0.4-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/typer-0.3.2-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/thinc-8.0.8-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/spacy_legacy-3.0.8-py2.py3-none-any.whl
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installati

In [2]:
!pip install --no-index --find-links="../input/commonlit-pkgs-offline-download" fastai==2.4.1

Looking in links: ../input/commonlit-pkgs-offline-download
Processing /kaggle/input/commonlit-pkgs-offline-download/fastai-2.4.1-py3-none-any.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/torchvision-0.10.0-cp37-cp37m-manylinux1_x86_64.whl
Processing /kaggle/input/commonlit-pkgs-offline-download/torch-1.9.0-cp37-cp37m-manylinux1_x86_64.whl
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.1
    Uninstalling torchvision-0.8.1:
      Successfully uninstalled torchvision-0.8.1
  Attempting uninstall: fastai
    Found existing installation: fastai 2.2.7
    Uninstalling fastai-2.2.7:
      Successfully uninstalled fastai-2.2.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflic

## Prepraring for inference

### Preparing the data for inference

In [3]:
import glob
import numpy
from functools import partial
from fastcore.all import *
from fastai.text.all import *

In [4]:
df_test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
df_test.head(3)

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,"My hope lay in Jack's promise that he would keep a bright light burning in the upper story to guide me on my course. On a clear night this light was visible from the village, but somehow or other I failed to take into account the state of the weather. The air was full of eddying flakes, which would render the headlight of a locomotive invisible a hundred yards distant. Strange that this important fact never occurred to me until I was fully a fourth of a mile from the village. Then, after looking in vain for the beacon light, the danger of my situation struck me, and I halted.\n""I am certai..."
1,f0953f0a5,NaN,NaN,"Dotty continued to go to Mrs. Gray's every night with the milk. Sometimes Katie went with her, and then they always paused a while under the acorn-tree and played ""King and Queen."" Dotty said she wished they could ever remember to bring their nipperkins, for in that case the milk would taste a great deal more like nectar. The ""nipperkins"" were a pair of handled cups which the children supposed to be silver, and which they always used at table.\nDotty knew she was doing wrong every time she played ""King and Queen."" She knew the milk was not hers, but Mrs. Gray's; still she said to herself, ..."
2,0df072751,NaN,NaN,"It was a bright and cheerful scene that greeted the eyes of Captain Raymond and his son as they entered the parlor of the adjacent cottage.\nIt was strictly a family gathering, yet the room was quite full. Mr. Dinsmore was there with his wife, his daughter Elsie and her children, Edward and Zoe, Elsie Leland with her husband and babe, Violet Raymond with her husband's two little girls, Lulu and Grace, and lastly Rosie and Walter.\nEverybody had a kindly greeting for the captain, and Violet's bright face grew still brighter as she made room for him on the sofa by her side.\n""We were beginni..."


In [5]:
df_test.shape

(7, 4)

Getting the list of ids. This would later be used to create the submission file

In [6]:
ids = df_test.id.to_list()
ids[:2]

['c0f722661', 'f0953f0a5']

Getting the excerpts to a list. This makes it easier to make predictions and later to create the submission file.

In [7]:
excerpts = df_test.excerpt.to_list()

## Loading the model

In [8]:
bkwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/back_final_model')
fwd_model = load_learner('../input/common-lit-forward-backward-training-on-ulmfit/forward_final_model')

In [9]:
bkwd_model.predict(excerpts[0])[0][0]

-1.6338351964950562

# Predictions

In [10]:
def ensamble(excerpts, bkwd_model, fwd_model):
    bkwd_preds = np.array([bkwd_model.predict(excerpt)[0][0] for excerpt in excerpts])
    fwd_preds = np.array([fwd_model.predict(excerpt)[0][0] for excerpt in excerpts])
    return np.mean([bkwd_preds, fwd_preds], axis=0)

In [11]:
predictions = ensamble(excerpts, bkwd_model, fwd_model)
#predictions = [float("{0:.2f}".format(pred.item())) for pred in ensamble(bkwd_model, fwd_model, excerpts)]
#predictions = [float("{0:.2f}".format(model.predict(excerpt)[0][0])) for excerpt in excerpts]

# Creating the submission file

Creating a new dataframe

In [12]:
submission_df = pd.DataFrame(list(zip(ids, predictions)), columns=['id', 'target'])

Saving the dataframe as csv for submission.

In [13]:
submission_df.head(2)

,id,target
0,c0f722661,-1.523477
1,f0953f0a5,-0.255050


In [14]:
submission_df.to_csv('submission.csv', index=False)